In [38]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
import pycrfsuite
from jinja2 import Template

import numpy as np
import pickle
import psycopg2

# Text Classification

In [3]:
# Load Pickle objects
count_vect = CountVectorizer()
tf_transformer = TfidfTransformer(use_idf=False)
clf = MultinomialNB()

open_file = open("classification.pkl", "rb")
count_vect, tfidf_transformer, clf = pickle.load(open_file)
open_file.close()

In [35]:
# text = 'Which patients are registered in R3500-AD-1906 ?'
text = 'What patients in R3500-AD-1906 are from Germany ?'
docs_new = []
docs_new.append(text)
print(docs_new)

['What patients in R3500-AD-1906 are from Germany ?']


In [36]:
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

In [37]:
categories = ['get_subjects', 'get_study']
#for doc, category in zip(docs_new, predicted):
#print(doc, category, categories[category-1])
sentence_category = categories[category-1]
print(doc, sentence_category)

Which patients are registered in R3500-AD-1906 ? get_subjects


# NER

In [16]:
def word2features(sent, i):
    word = sent[i][0]
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        ]
    if i > 0:
        word1 = sent[i-1][0]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
        ])
    else:
        features.append('BOS')
        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
        ])
    else:
        features.append('EOS')
                
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

def sent2tokens(sent):
    return [token for token, label in sent]

In [17]:
tagger = pycrfsuite.Tagger()
tagger.open('test.crfsuite')

In [39]:
print(text.split())
ner_input = [(tok, 'O') for tok in text.split()]
print(ner_input)
ner_output = tagger.tag(sent2features(ner_input))
print(ner_output)

['What', 'patients', 'in', 'R3500-AD-1906', 'are', 'from', 'Germany', '?']
[('What', 'O'), ('patients', 'O'), ('in', 'O'), ('R3500-AD-1906', 'O'), ('are', 'O'), ('from', 'O'), ('Germany', 'O'), ('?', 'O')]
['O', 'O', 'O', 'STUDYID', 'O', 'O', 'COUNTRY', 'O']


# SQL Generator

In [40]:
sql_template = {'get_subjects': 'select subjectid from subject_listing where',
                'get_study': 'select studyid from study_listing where'}

conditions = []
for word, label in zip(ner_input, ner_output):
    #print(word[0], label)
    if label != str("O"):
        conditions.append(str(label) + '=' + str('\'') + str(word[0]) + str('\''))

In [41]:
final_sql_condition = " AND ".join(conditions)
sql_query = str(sql_template[sentence_category]) + " " + final_sql_condition
print(sql_query)

select subjectid from subject_listing where STUDYID='R3500-AD-1906' AND COUNTRY='Germany'


# Query Database

In [42]:
# Connect to your postgres DB
conn = psycopg2.connect("dbname=postgres user=postgres password=postgres")

# Open a cursor to perform database operations
cur = conn.cursor()

# Execute a query
cur.execute(sql_query)

# Retrieve query results
records = cur.fetchall()

# Close the connection
conn.close()

In [43]:
print(records)

[('1',), ('2',)]
